# CDAT Migration Regression Testing Notebook (`.nc` files)

This notebook is used to perform regression testing between the development and
production versions of a diagnostic set.

## How it works

It compares the relative differences (%) between ref and test variables between
the dev and `main` branches.

## How to use

PREREQUISITE: The diagnostic set's netCDF stored in `.json` files in two directories
(dev and `main` branches).

1. Make a copy of this notebook under `auxiliary_tools/cdat_regression_testing/<DIR_NAME>`.
2. Run `mamba create -n cdat_regression_test -y -c conda-forge "python<3.12" xarray netcdf4 dask pandas matplotlib-base ipykernel`
3. Run `mamba activate cdat_regression_test`
4. Update `SET_DIR` and `SET_NAME` in the copy of your notebook.
5. Run all cells IN ORDER.
6. Review results for any outstanding differences (>=1e-5 relative tolerance).
   - Debug these differences (e.g., bug in metrics functions, incorrect variable references, etc.)


## Setup Code


In [11]:
from collections import defaultdict
import glob
from typing import Tuple

import numpy as np
import xarray as xr

from e3sm_diags.derivations.derivations import DERIVED_VARIABLES

DEV_DIR = "843-migration-phase3"
DEV_PATH = f"/global/cfs/cdirs/e3sm/www/cdat-migration-fy24/{DEV_DIR}/"
DEV_GLOB = sorted(glob.glob(DEV_PATH + "**/**/*.nc"))
DEV_NUM_FILES = len(DEV_GLOB)

MAIN_DIR = "v2_9_0_all_sets_model_vs_model"
MAIN_PATH = f"/global/cfs/cdirs/e3sm/www/cdat-migration-fy24/main/{MAIN_DIR}/"
MAIN_GLOB = sorted(glob.glob(MAIN_PATH + "**/**/*.nc"))
MAIN_NUM_FILES = len(MAIN_GLOB)

In [12]:
def _check_if_files_found():
    if DEV_NUM_FILES == 0 or MAIN_NUM_FILES == 0:
        raise IOError(
            "No files found at DEV_PATH and/or MAIN_PATH. "
            f"Please check {DEV_PATH} and {MAIN_PATH}."
        )


def _check_if_matching_filecount():
    if DEV_NUM_FILES != MAIN_NUM_FILES:
        raise IOError(
            "Number of files do not match at DEV_PATH and MAIN_PATH "
            f"({DEV_NUM_FILES} vs. {MAIN_NUM_FILES})."
        )

    print(f"Matching file count ({DEV_NUM_FILES} and {MAIN_NUM_FILES}).")


def _check_if_missing_files():
    missing_dev_files = []
    missing_main_files = []

    for fp_main in MAIN_GLOB:
        fp_dev = fp_main.replace(MAIN_PATH, DEV_PATH)

        if fp_dev not in DEV_GLOB:
            missing_dev_files.append(fp_dev)

    for fp_dev in DEV_GLOB:
        fp_main = fp_dev.replace(DEV_PATH, MAIN_PATH)

        if fp_main not in MAIN_GLOB:
            missing_main_files.append(fp_main)

    return missing_dev_files, missing_main_files

In [22]:
def _get_relative_diffs():
    # We are mainly focusing on relative tolerance here (in percentage terms).
    atol = 0
    rtol = 1e-5

    results = {
        "missing_files": [],
        "matching_files": [],
        "mismatch_errors": [],
        "not_equal_errors": [],
        "key_errors": [],
    }

    for fp_main in MAIN_GLOB:
        if "_diff.nc" not in fp_main:
            fp_dev = fp_main.replace("examples-main", "examples-dev")

            print("Comparing:")
            print(f"    * {fp_dev}")
            print(f"    * {fp_main}")

            try:
                ds1 = xr.open_dataset(fp_dev)
                ds2 = xr.open_dataset(fp_main)
            except FileNotFoundError as e:
                print(f"    {e}")

                if isinstance(e, FileNotFoundError):
                    results["missing_files"].append(fp_dev)

                continue

            var_key = fp_main.split("-")[-3]
            # for 3d vars such as T-200
            var_key.isdigit()
            if var_key.isdigit():
                var_key = fp_main.split("-")[-4]

            print(f"    * var_key: {var_key}")

            dev_data = _get_var_data(ds1, var_key)
            main_data = _get_var_data(ds2, var_key)

            if dev_data is None or main_data is None:
                print("    * Could not find variable key in the dataset(s)")
                continue

            try:
                np.testing.assert_allclose(
                    dev_data,
                    main_data,
                    atol=atol,
                    rtol=rtol,
                )
                results["matching_files"].append(fp_main)
            except (KeyError, AssertionError) as e:
                msg = str(e)

                print(f"    {msg}")
                if "mismatch" in msg:
                    results["mismatch_errors"].append(fp_main)
                elif "Not equal to tolerance" in msg:
                    results["not_equal_errors"].append(fp_main)
            else:
                print(f"    * All close and within relative tolerance ({rtol})")

    return results


def _get_var_data(ds: xr.Dataset, var_key: str) -> np.ndarray:
    """Get the variable data using a list of matching keys.

    The `main` branch saves the dataset using the original variable name,
    while the dev branch saves the variable with the derived variable name.
    The dev branch is performing the expected behavior here.

    Parameters
    ----------
    ds : xr.Dataset
        _description_
    var_key : str
        _description_

    Returns
    -------
    np.ndarray
        _description_
    """

    data = None

    var_keys = DERIVED_VARIABLES[var_key].keys()
    var_keys = [var_key] + list(sum(var_keys, ()))

    for key in var_keys:
        if key in ds.data_vars.keys():
            data = ds[key].values
            break

    return data

## 1. Check for matching and equal number of files


In [23]:
_check_if_files_found()

In [24]:
missing_dev_files, missing_main_files = _check_if_missing_files()

In [25]:
len(missing_dev_files)

134

In [26]:
len(missing_main_files)

290

In [27]:
_check_if_matching_filecount()

OSError: Number of files do not match at DEV_PATH and MAIN_PATH (2120 vs. 1964).

## 2 Compare the netCDF files between branches

- Compare "ref" and "test" files
- "diff" files are ignored because getting relative diffs for these does not make sense (relative diff will be above tolerance)


In [28]:
_get_relative_diffs()

Comparing:
    * /global/cfs/cdirs/e3sm/www/cdat-migration-fy24/main/v2_9_0_all_sets_model_vs_model/annual_cycle_zonal_mean/CERES-EBAF-TOA-v4.1/ceres_ebaf_toa_v4.1-ALBEDO-Annual-Cycle_ref.nc
    * /global/cfs/cdirs/e3sm/www/cdat-migration-fy24/main/v2_9_0_all_sets_model_vs_model/annual_cycle_zonal_mean/CERES-EBAF-TOA-v4.1/ceres_ebaf_toa_v4.1-ALBEDO-Annual-Cycle_ref.nc
    * var_key: ALBEDO
    * All close and within relative tolerance (1e-05)
Comparing:
    * /global/cfs/cdirs/e3sm/www/cdat-migration-fy24/main/v2_9_0_all_sets_model_vs_model/annual_cycle_zonal_mean/CERES-EBAF-TOA-v4.1/ceres_ebaf_toa_v4.1-ALBEDO-Annual-Cycle_test.nc
    * /global/cfs/cdirs/e3sm/www/cdat-migration-fy24/main/v2_9_0_all_sets_model_vs_model/annual_cycle_zonal_mean/CERES-EBAF-TOA-v4.1/ceres_ebaf_toa_v4.1-ALBEDO-Annual-Cycle_test.nc
    * var_key: ALBEDO
    * All close and within relative tolerance (1e-05)
Comparing:
    * /global/cfs/cdirs/e3sm/www/cdat-migration-fy24/main/v2_9_0_all_sets_model_vs_model/annu

KeyError: 'lhflx'

### Results

All files are within rtol 1e-5, so the changes should be good to go. There are two
that seem like they are about rtol, but they aren't (still around 1e-5).
